In [1]:
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.decomposition import PCA
plt.rcParams['font.sans-serif']=['SimHei'] #图中文字体设置为黑体
plt.rcParams['axes.unicode_minus']=False
from tqdm.notebook import tqdm
from sklearn.metrics import silhouette_score
import scipy
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import math
from scipy.stats import entropy

In [2]:
def read_data(file_name):
    df = pd.read_csv(file_name)
    df = df.drop(labels=['BASIC_entity_name', 'BASIC_year'], axis=1)
    fields_df = pd.read_excel('IDOU_05_事件触发_03_大小类事件映射表_v2.0.xlsx')
    rename_dict = {}
    for i, row in fields_df.iterrows():
        rename_dict[row['大类事件代码'] + '_' + row['小类事件代码']] = row['名称']
    df = df.rename(columns=rename_dict)
    if 'REVENUEAB_test' in df.columns:
        df = df.drop(labels='REVENUEAB_test', axis=1)
    return df

def gen_value_count_df(df):
    value_count_dict = {0:[], 1:[], 2:[]}
    for col in df.columns:
        value_count = df.value_counts(col, normalize=True)
        for level in range(3):
            try:
                value_count_dict[level].append(value_count.loc[level])
            except:
                value_count_dict[level].append(0)
    return pd.DataFrame(value_count_dict, index=df.columns).transpose()

def calc_sim(cols_sampled):
    vec1 = value_count_df_part[cols_sampled].to_numpy().flatten()
    vec2 = value_count_df_all[cols_sampled].to_numpy().flatten()
    return vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def calc_kl(cols_sampled):
    kl_divergences = []
    A = value_count_df_part[cols_sampled].to_numpy().transpose()
    B = value_count_df_all[cols_sampled].to_numpy().transpose()
    for i in range(A.shape[0]):
        kl_divergences.append(entropy(A[i], B[i]))
    return -np.mean(kl_divergences)

def drop_duplicate_index(df):
    all_index = df.index.to_list()
    all_index_split = [set(i.split(',')) for i in all_index]
    for i in tqdm(range(len(all_index_split))):
        if all_index_split[i] in all_index_split[:i]:
            df.drop(all_index[i], inplace=True)
    return df

def drop_dup_results(keys, values):
    for k in tqdm(list(set(keys))):
        k_count = keys.count(k)
        if k_count > 1:
            for _ in range(k_count - 1):
                idx = keys.index(k)
                keys.pop(idx)
                values.pop(idx)
    return keys, values

In [3]:
df_all = read_data('舞弊案例全市场数据2014-2021.csv')
fields_df = pd.read_excel('IDOU_05_事件触发_03_大小类事件映射表_v2.0.xlsx')
value_count_df_all = gen_value_count_df(df_all)

In [4]:
value_count_df_all['资产质量异常-大额长期应收款']

0    0.994614
1    0.003218
2    0.002168
Name: 资产质量异常-大额长期应收款, dtype: float64

In [7]:
for df_part_name in tqdm(['all_samples', '舞弊案例数据库']):
    for k in [5, 10]:
        for similarity in [calc_sim, calc_kl]:
            df_part = read_data(f'{df_part_name}.csv')
            value_count_df_part = gen_value_count_df(df_part)
            value_count_df_all = value_count_df_all[value_count_df_part.columns]
            
            result_keys = []
            result_values = []
            for x in tqdm(range(10000)):
                cols_sampled = random.sample(value_count_df_part.columns.to_list(), k)
                sim = similarity(cols_sampled)
                for y in range(20):
                    cols_to_drop = random.sample(cols_sampled, random.randint(1, k-2))
                    cols_sampled_after_drop = [c for c in cols_sampled if c not in cols_to_drop]
                    sim_after_drop = similarity(cols_sampled_after_drop)
                    if sim_after_drop < sim:
                        if set(cols_sampled_after_drop) not in result_keys:
                            result_keys.append(set(cols_sampled_after_drop))
                            result_values.append(sim_after_drop)
            result_dict = {}
            for i in range(len(result_keys)):
                result_dict[','.join(list(result_keys[i]))] = result_values[i]
            result_df = pd.DataFrame(result_dict, index=['sim']).T.sort_values(by='sim')
            result_df.to_excel(f'分组相似度 k={k} 舞弊样本={df_part_name} 相似度指标={similarity.__name__}.xlsx')

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]